In [10]:
%%capture
try:
    from pip import main as pipmain
except:
    from pip._internal import main as pipmain
packages = ['twython']
pipmain(['install'] + packages)

In [11]:
%%capture
from twython import Twython

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import re
import random

In [12]:
from auth import (
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

twitter = Twython(
    consumer_key,
    consumer_secret,
    access_token,
    access_token_secret
)

In [13]:
PRINT_LIMIT = 50
FOLLOWER_SAMPLE_LIMIT = 20 # number of followers to randomly sample
WORD_FREQ_LIMIT = 10 # return this number of topics that are most freq

In [14]:
user = "AndrewYang"

In [15]:
followers = twitter.get_followers_ids(screen_name = user)

limit = 0
for follower_id in followers['ids']:
    print('Follower Id: {0} '.format(follower_id))
    limit += 1
    if limit >= PRINT_LIMIT:
        break

Follower Id: 1066754695912349696 
Follower Id: 3094679364 
Follower Id: 4898817859 
Follower Id: 1046973263652499456 
Follower Id: 826279704764125184 
Follower Id: 56789243 
Follower Id: 1025216100718583809 
Follower Id: 1187231128282947584 
Follower Id: 2206178504 
Follower Id: 2811529981 
Follower Id: 2214471943 
Follower Id: 1187230717215940610 
Follower Id: 2361794225 
Follower Id: 2460219890 
Follower Id: 1187227948199370752 
Follower Id: 728381963761917952 
Follower Id: 991509045101056000 
Follower Id: 314851797 
Follower Id: 1187094020624977920 
Follower Id: 1115134837 
Follower Id: 1168595858516074498 
Follower Id: 2682096483 
Follower Id: 761434193993854976 
Follower Id: 922107187 
Follower Id: 1180084653979586562 
Follower Id: 841037012 
Follower Id: 2377608512 
Follower Id: 1187228886834257920 
Follower Id: 835799316 
Follower Id: 1618391833 
Follower Id: 454538051 
Follower Id: 2381104249 
Follower Id: 14817882 
Follower Id: 1017675294830878720 
Follower Id: 116685753213829

In [16]:
def get_tweet_data(results):
    tweets = []
    for tweet in results:
        tweet_map = {"text": tweet['text']}
        tweet_hashtags = []
        tweet_urls = []
        tweet_user_mentions = []
        
        entities = tweet['entities']
    
        for hashtags in entities['hashtags']:
            tweet_hashtags.append(hashtags['text'])
        for urls in entities['urls']:
            tweet_urls.append(urls['url'])
        try:
            for media in entities['media']:
                tweet_urls.append(media['url'])
        except:
            pass
        for users in entities['user_mentions']:
            tweet_user_mentions.append(users['screen_name'])
            
        tweet_map["hashtags"] = tweet_hashtags
        tweet_map["urls"] = tweet_urls
        tweet_map["user_mentions"] = tweet_user_mentions
        tweets.append(tweet_map)
        
    return tweets

In [19]:
def get_tweets_data_from(user_id, results):
    try: 
        result = twitter.get_user_timeline(user_id = user_id)
        results.extend(result)
        return get_tweet_data(result), results
    except:
        return [], results

In [20]:
def get_tweets_data_and_results(user_ids):
    tweets_data = []
    results = []
    
    samples = user_ids
    if len(user_ids) > FOLLOWER_SAMPLE_LIMIT:
        samples = random.sample(user_ids, FOLLOWER_SAMPLE_LIMIT)

    for f in samples:
        result, results = get_tweets_data_from(f, results)
        tweets_data.extend(result)
            
    return tweets_data, results
    
tweets_data, results = get_tweets_data_and_results(followers['ids'])

In [21]:
tweets_data

[{'text': 'RT @utahjazz: All-Star is coming back to SLC in 2023 ❄️🏀🌟 https://t.co/yHaX78p8Id',
  'hashtags': [],
  'urls': ['https://t.co/yHaX78p8Id'],
  'user_mentions': ['utahjazz']},
 {'text': '@jazzteamstore @dalystexxxas',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['jazzteamstore', 'dalystexxxas']},
 {'text': 'RT @jazzteamstore: ⚡️TICKET GIVEAWAY⚡️\nWe’re giving away 2 tickets to tonight’s home opener vs. Oklahoma City Thunder!\n\nTo Enter:\n1️⃣Follow…',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['jazzteamstore']},
 {'text': '@MLozada 😡',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['MLozada']},
 {'text': 'RT @summit1g: Giveaway time! Corsair hitting someone up with a new K70 MK.2 SE\n\nTo Enter: \n- Retweet and Like this post\n- Follow @Summit1G…',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['summit1g', 'summit1g']},
 {'text': '@vivintarena @utahjazz @trailblazers @dalystexxxas',
  'hashtags': [],
  'urls': [],
  'user_mentions': ['vivintarena',
 

[NLTK Stopwords](https://pythonspot.com/nltk-stop-words/)

In [22]:
STOPWORDS = stopwords.words('english') + stopwords.words('spanish')
# TODO: decide whether or not to remove tweets that are from retweets altogether
more_stop_words = ['rt']
STOPWORDS += more_stop_words
STOPWORDS

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [23]:
def word_extraction(tweet_dictionary, stopwords = STOPWORDS):  
    sentence = tweet_dictionary['text']
    hashtags = ['#' + s for s in tweet_dictionary['hashtags']]
    urls = tweet_dictionary['urls']
    user_mentions = ['@' + s for s in tweet_dictionary['user_mentions']]
    
    ignore = hashtags + urls + user_mentions

    words = sentence.split()  
    
    cleaned_text = []
    
    for word in words:
        cleaned_word = re.sub("[^\w]", "",  word)
        if (not word in ignore) and (not word.lower() in stopwords) and (word.isalpha()) and (len(cleaned_word) > 0):
            cleaned_text.append(cleaned_word.lower())
    return cleaned_text

In [24]:
test_tweet = tweets_data[0]
print(word_extraction(test_tweet))
print(test_tweet)

['coming', 'back', 'slc']
{'text': 'RT @utahjazz: All-Star is coming back to SLC in 2023 ❄️🏀🌟 https://t.co/yHaX78p8Id', 'hashtags': [], 'urls': ['https://t.co/yHaX78p8Id'], 'user_mentions': ['utahjazz']}


In [25]:
tweets_text = []
for tweet in tweets_data:
    tweets_text.extend(word_extraction(tweet))
print("Extracted words:", tweets_text)

Extracted words: ['coming', 'back', 'slc', 'giving', 'away', 'tickets', 'home', 'opener', 'oklahoma', 'city', 'giveaway', 'corsair', 'hitting', 'someone', 'new', 'retweet', 'like', 'post', 'follow', 'flash', 'win', 'tickets', 'vs', 'preseason', 'game', 'dinner', 'hires', 'big', 'h', 'giving', 'away', 'tickets', 'preseason', 'game', 'sacramento', 'fortnite', 'demonized', 'children', 'turn', 'back', 'normal', 'gaming', 'give', 'someone', 'random', 'retweets', 'pay', 'following', 'dm', 'ends', 'win', 'tickets', 'preseason', 'game', 'monday', 'october', 'dinner', 'guys', 'could', 'please', 'help', 'donate', 'even', 'retweet', 'tweet', 'thank', 'enjoying', 'season', 'apex', 'lot', 'battlepass', 'check', 'give', 'cash', 'someone', 'random', 'retweets', 'beautiful', 'afternoon', 'following', 'dm', 'boys', 'back', 'win', 'suite', 'tickets', 'first', 'preseason', 'game', 'season', 'battle', 'pass', 'winners', 'announced', 'retweet', 'tweet', 'must', 'following', 'give', 'someone', 'random', 're

In [26]:
tweets_hashtags = []
for tweet in tweets_data:
        tweets_hashtags.extend(tweet['hashtags'])
print("Extracted hashtags:", tweets_hashtags)

Extracted hashtags: ['VivintArena', 'PS5', 'FreePizzaFriday', 'photographer', 'vancouverprofessional', 'canadianphotographer', 'photographer', 'vancouverprofessional', 'vancouverbusiness', 'photoboxvancouver', 'vancouverevent', 'Vancouver', 'happyholidays', 'vancouverphotography', 'photographer', 'vancovuerfashion', 'photoshoot', 'photography', 'vancouver', 'vancouverbusiness', 'activewear', 'lookbook', 'vancouverprofessional', 'busybusy', 'MakeFirestormFree', 'magic', 'twitch', 'StreamingLive', 'jackboxpartypack6', 'twitch', 'streaminglive', 'SuperMarioMaker2', 'twitch', 'streaminglive', 'supermariomaker2', 'twitch', 'streaminglive', 'SuperMarioMaker2', 'twitch', 'streaminglive', 'SuperMarioMaker2', 'twitch', 'JackboxPartyPack6', 'JackboxStream', 'twitch', 'streaminglive', 'SuperMarioMaker2', 'twitch', 'streaminglive', 'SuperMarioMaker2', 'twitch', 'streaminglive', 'marbles', 'twitch', 'arcoiris', 'beavertonschool', 'NewProfilePic', 'WORLDSERIES', 'GIVEAWAY', 'FACEMILKS', 'TMobileTues

In [27]:
def get_freq_map(my_list): 
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    return freq

def get_top_freq(my_list, top_count):
    freq = get_freq_map(my_list)
    freq = sorted(freq.items(), key=lambda item: item[1], reverse = True)
    
    if len(freq) <= top_count:
        return freq
    else:
        return freq[:10]

In [28]:
tweets_text_freq = get_top_freq(tweets_text, WORD_FREQ_LIMIT)
print("top {0} most freq words:".format(WORD_FREQ_LIMIT))
tweets_text_freq

top 10 most freq words:


[('going', 15),
 ('live', 9),
 ('win', 8),
 ('game', 8),
 ('get', 8),
 ('new', 7),
 ('retweet', 7),
 ('great', 7),
 ('super', 7),
 ('tickets', 6)]

In [29]:
tweets_hashtags_freq = get_top_freq(tweets_hashtags, WORD_FREQ_LIMIT)
print("top {0} most freq hashtags:".format(WORD_FREQ_LIMIT))
tweets_hashtags_freq

top 10 most freq hashtags:


[('twitch', 10),
 ('streaminglive', 7),
 ('SuperMarioMaker2', 5),
 ('photographer', 3),
 ('vancouverprofessional', 3),
 ('vancouverbusiness', 2),
 ('VivintArena', 1),
 ('PS5', 1),
 ('FreePizzaFriday', 1),
 ('canadianphotographer', 1)]